In [39]:
import urllib.request
import json
import numpy as np
import pandas as pd
import time
import re
from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from nltk import FreqDist
import requests
from bs4 import BeautifulSoup
import pickle

## billboard

In [4]:
# may 2019 - present
response = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Abillboard.com%20AND%20discoverDate%3A%20%5B2019-05-01%20TO%20*%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", 
    headers={"Authorization":"YOUR API KEY"})
results_2019_1 = response.json()

In [5]:
#feb - april 2019
response1 = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Abillboard.com%20AND%20discoverDate%3A%20%5B2019-02-01%20TO%202019-04-30%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", 
    headers={"Authorization":"YOUR API KEY"})
results_2019_2 = response1.json()

In [6]:
#jan 2019
response2 = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Abillboard.com%20AND%20discoverDate%3A%20%5B2019-01-01%20TO%202019-01-31%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", 
    headers={"Authorization":"YOUR API KEY"})
results_2019_3 = response2.json()

In [7]:
# everything in 2018
response3 = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Abillboard.com%20AND%20discoverDate%3A%20%5B2018-01-01%20TO%202018-12-31%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", headers={"Authorization":"YOUR API KEY"})
results_2018_1 = response3.json()

In [19]:
df_bb = pd.DataFrame(results_2019_1+results_2019_2+results_2019_3+results_2018_1)

In [21]:
df_bb.to_pickle("./df_bb.pkl")

## digital music news

In [9]:
response4 = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Adigitalmusicnews.com%20AND%20discoverDate%3A%20%5B2018-01-01%20TO%20*%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", headers={"Authorization":"YOUR API KEY"})
results_2018_2019 = response4.json()

In [22]:
df_dmn = pd.DataFrame(results_2018_2019)

In [23]:
df_dmn.to_pickle("./df_dmn.pkl")

## music business worldwide

In [10]:
#everything in 2020
response5 = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Amusicbusinessworldwide.com%20AND%20discoverDate%3A%20%5B2020-02-15%20TO%20*%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", headers={"Authorization":"YOUR API KEY"})
results_mbw_feb2 = response5.json()

In [11]:
response6 = requests.get("https://api.newsriver.io/v2/search?query=website.domainName%3Amusicbusinessworldwide.com%20AND%20discoverDate%3A%20%5B2020-02-01%20TO%202020-02-14%5D&sortBy=discoverDate&sortOrder=DESC&limit=100", headers={"Authorization":"YOUR API KEY"})
results_mbw_feb1 = response6.json()

In [24]:
df_mbw_2020 = pd.DataFrame(results_mbw_feb2+results_mbw_feb1)

In [25]:
df_mbw_2020.to_pickle("./df_mbw_2020.pkl")

### define dates for all 2018&2019 posts

In [12]:
begin_date = []
for year in ["2018","2019"]:
    for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
            date_range1 = (year + '-' + month + '-' "01")
            begin_date.append(date_range1)
    for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
            date_range2 = (year + '-' + month + '-' + "16")
            begin_date.append(date_range2)


In [13]:
begin_date = sorted(begin_date)

In [14]:
end_date = []
for year in ["2018","2019"]:
    for month in ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]:
            date_range1 = (year + '-' + month + '-' "15")
            end_date.append(date_range1)
    for month in ["01", "03", "05", "07", "08", "10", "12"]:
            date_range2 = (year + '-' + month + '-' "31")
            end_date.append(date_range2)
    for month in ["02"]:
            date_range3 = (year + '-' + month + '-'  "28")
            end_date.append(date_range3)
    for month in ["04", "06", "09", "11"]:
            date_range4 = (year + '-' + month + '-'  "30")
            end_date.append(date_range4)

In [15]:
end_date = sorted(end_date)

In [16]:
all_dates = list(zip(begin_date, end_date))

### getting articles from all these dates

In [17]:
def get_articles(all_dates):
    all_articles = []
    for begin_date, end_date in all_dates:
        url = "https://api.newsriver.io/v2/search?query=website.domainName%3Amusicbusinessworldwide.com%20AND%20discoverDate%3A%20%5B"+ str(begin_date) + "%20TO%20"+ str(end_date) + "%5D&sortBy=discoverDate&sortOrder=DESC&limit=100"
        response = requests.get(url , headers={"Authorization":"YOUR API KEY"}) 
        results = response.json()
        all_articles.append(results)
    return all_articles

In [18]:
data_mbw = get_articles(all_dates)

In [74]:
articles = []
for item in data_mbw:
    if item != []:
        for item in item:
            articles.append(item)

In [75]:
df_mbw_2019 =  pd.DataFrame(articles)

In [76]:
df_mbw_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 12 columns):
id                1820 non-null object
publishDate       1816 non-null object
discoverDate      1820 non-null object
title             1820 non-null object
language          1820 non-null object
text              1820 non-null object
structuredText    1820 non-null object
url               1820 non-null object
elements          1820 non-null object
website           1820 non-null object
metadata          1820 non-null object
score             1820 non-null float64
dtypes: float64(1), object(11)
memory usage: 170.8+ KB


In [77]:
df_mbw_2019.to_pickle("./df_mbw_2019.pkl")